### File organization
Python code is stored in pure/py and its subdirectories. pure/data and pure/obj contains folders which store scan data in the form of `.pkl` (serialized numpy arrays) or `.npy` (binary numpy) files, to be used with python. Folder and file names containing the prefix "1D" denote a one dimensional scan, "2D" denotes a two dimensional scan, and those without any aforementioned prefixes denote angle dependence scans (stationary transducer with rotating metal sample). Images of scan results are placed in subdirectories of pure/scans.

## Angle Dependence
1. How do you set up the experiment?

The setup consists of a metal sample with a micrometer built-in which tilts the sample in a large bucket of water. To increase repeatability, we mark the position of the sample on the bottom surface of the bucket with tape and other pieces of metal.

We use a protractor to measure the micrometer reading corresponding to every degree of inclination, starting at 0 degrees and going up to 15 degrees. We perform a linear fit on this data using `scipy.optimize.curve_fit()` and a linear model function $f(\theta) = a\theta+b$, where $\theta$ is the angle of inclination and $f$ is the micrometer reading.

2. How do you collect data for the angle dependence experiment?

Using the angle to micrometer reading table, we rotate the micrometer to fit a certain angle, place it in the bucket of 
water, let the water ripples settle down, then save the oscilloscope data. How do we save the oscilloscope data? It is best to use the python console and import `scope` module. Create the output directory, and create a Scope object, passing the directory (as a numpy path) as a parameter. Finally, run `scope.Scope.grab()` to make a single measurement, and the signal will be saved as an `.npy` file in the specified directory.

In [ ]:
import py.scope as scope

out_folder = "C:\\Users\\dionysius\\Desktop\\PURE\\pure\\data\\TEST_TRANS"
o = scope.Scope(out_folder, filename='test_scope')

In [ ]:
o.grab()

3. How do I analyze a dataset obtained in the angle dependence experiment?

You will need to import `py.main`. First, define a variable for the path to the `.npy` data obtained from the scope. Next, instantiate a Transducer object, and call as shown below. Then, call its `write_all()` method and see if the correct peaks have been selected by viewing the figures saved in the subfolder `signal`.

4. How do I do a B-Scan on one angle dependence experiment dataset?
First load data into a variable using `load_obj` from `main`. Pass the name (listed in '/obj') of the dataset into the function as a string.

5. How do I perform a 1D or 2D scan?

Look at `scanning.py`, change the variable `SCAN_FOLDER` at the top to the appropriate name, and in the main program call the class `Scan` with parameters `DIMENSIONS` and `START_POS`. `DIMENSIONS` takes a tuple of real values representing the scanning grid in units of metres. `START_POS` can be 'top right', 'bottom right', etc. Grid formed is an TxMxN numpy matrix

## SAFT


In [2]:
import numpy as np
import threading
from numpy import \
    reshape as np_reshape, \
    power as np_power, \
    sqrt as np_sqrt, \
    argmin as np_argmin, \
    abs as np_abs, \
    sum as np_sum
from os import getcwd
from os.path import join, dirname
import pickle
import matplotlib.pyplot as plt

`numpy` is used for array and general number operations. We alias some `numpy` functions to decrease overhead in the main loop of the program. `threading` is used for multithreading the main program. The functions called from `os` are for system directories and paths to file. `pickle` is used to serialize objects into files. `matplotlib` is used for plotting graphs. `

In [3]:
global min_step, c_0, DEFAULT_ARR_FOLDER
global xarr, FD, SD, pbar, T, V, L, T_COMPARE, PRE_OUT, POST_OUT, xni
FOLDER_NAME = "1D-3FOC7in0DEG - 1"
DEFAULT_ARR_FOLDER = join(dirname(getcwd()), "data", FOLDER_NAME)
FOCAL_DEPTH = 0.0381  # 1.5 inch in metres
min_step = 4e-4
c_0 = 1498  # water

Here we establish folder paths, and define parameters and constants.

In [ ]:
def load_arr(output_folder=DEFAULT_ARR_FOLDER):
    ftarr = join(output_folder, "tarr.pkl")
    fvarr = join(output_folder, "varr.pkl")
    with open(ftarr, 'rb') as rd:
        tarr = pickle.load(rd)
    with open(fvarr, 'rb') as rd:
        varr = pickle.load(rd)
    return tarr, varr


def find_nearest(array, value):
    array = np.asarray(array, dtype=float)
    return (np.abs(array - value)).argmin()

`load_arr()` outputs time and voltage (2D) arrays. `find_nearest()` finds the index in an array best corresponding to a value.

In [ ]:
tarr, varr = load_arr()
tarr = tarr[:, 0, :]
varr = varr[:, 0, :]
ZERO = find_nearest(tarr[:, 0], 0)
T = tarr[ZERO:, 0]  # 1D, time columns all the same
V = varr[ZERO:, :]  # 2D
FD = find_nearest(T, 2*FOCAL_DEPTH/c_0)  # focal depth
# SD = find_nearest(T, 2*SAMPLE_DEPTH/c_0) + 1  # sample depth
SD = len(T)-1
OFFSET = T[FD]
L = np.shape(V)[1]
PRE = np.flip(V[:FD, :], axis=0)
PRE_T = np.flip(T[:FD], axis=0)
PRE_OUT = np.empty(np.shape(PRE))
POST = V[FD:SD, :]
POST_T = T[FD:SD]
POST_OUT = np.empty(np.shape(POST))
xarr = np.linspace(-L/2, L/2, L)*min_step
xni = np.arange(0, L, 1)
tstep = np.mean(T[1:]-T[:-1])

In [ ]:
def main(xi):  # xi is imaging index/position
    x = xarr[xi]  # x is imaging DISTANCE
    ti = 0  # imaging pixel for time/ z direction
    while ti < SD:
        z = T[ti]*c_0/2
        z2 = np_power(z, 2)  # distance, squared
        ind = (2/c_0)*np_sqrt(np_power(x-xarr[xni], 2)
                              + z2)
        zi = (ind/tstep).astype(int)
        if ti < FD:  # PRE
            PRE_OUT[ti, xi] = np_sum(V[zi[zi < FD], xni[zi < FD]])
        if ti >= FD:  # POST
            POST_OUT[ti-FD, xi] = np_sum(V[zi[zi < SD], xni[zi < SD]])
        ti += 1

In [ ]:
if __name__ != '__main__':
    # Parallel processing
    jobs = []
    print("Append")
    for i in range(L):
        jobs.append(threading.Thread(target=main, args=(i,)))
    print("Starting")
    for job in jobs:
        job.start()
    print("Joining")
    for job in jobs:
        job.join()
    print("Stitching")
    PRE_OUT = np.flip(PRE_OUT, axis=0)
    STITCHED = np.vstack((PRE_OUT, POST_OUT))
    pickle.dump(STITCHED, open(join(DEFAULT_ARR_FOLDER,
                                    "SAFT-{}-test.pkl"
                                    .format(FOLDER_NAME)), "wb"))

In [ ]:
fig = plt.figure(figsize=[10, 10])
plt.imshow(STITCHED[:, :], aspect='auto', cmap='hot')
plt.colorbar()
plt.show()